# c8ctl End-to-End Operations Guide

This notebook demonstrates comprehensive end-to-end operations using the Camunda 8 CLI (`c8ctl`).

## Prerequisites

1. **Node.js v22 LTS** installed
2. **Camunda 8 cluster** running at `localhost:8080`
3. **c8ctl CLI** installed globally or available via `node src/index.ts`
4. **Jupyter with Node.js kernel** (tslab or IJavascript)

## Setup Instructions

### Install Jupyter Notebook Kernel for Node.js

```bash
# Install tslab globally
npm install -g tslab
tslab install --version

# Or use IJavascript
npm install -g ijavascript
ijsinstall
```

### Start Camunda 8 Locally

```bash
# Using Docker Compose (from the c8ctl repo)
cd assets/c8/8.8
docker compose --profile elasticsearch up -d

# Wait for Camunda to be ready
curl -u demo:demo http://localhost:8080/v2/topology
```

### Run This Notebook

```bash
jupyter notebook examples/e2e-operations.ipynb
```

## 1. Environment Setup and Verification

First, let's verify our environment and check connectivity to the Camunda cluster.

In [ ]:
// Import required modules
const { execSync } = require('child_process');
const fs = require('fs');
const path = require('path');

// Helper function to execute c8ctl commands
function c8(command, options = {}) {
    const cmd = `node ${path.join(__dirname, '..', 'src', 'index.ts')} ${command}`;
    try {
        const result = execSync(cmd, { 
            encoding: 'utf8',
            stdio: options.silent ? 'pipe' : 'inherit',
            ...options 
        });
        return result;
    } catch (error) {
        if (!options.ignoreErrors) throw error;
        return error.stdout || '';
    }
}

console.log('✓ Environment setup complete');

In [ ]:
// Check Node.js version
console.log('Node.js version:', process.version);
console.log('Platform:', process.platform);

// Verify we have v22 LTS
const majorVersion = parseInt(process.version.slice(1).split('.')[0]);
if (majorVersion < 22) {
    console.warn('⚠️  Warning: Node.js v22 or higher is recommended');
} else {
    console.log('✓ Node.js version is compatible');
}

In [ ]:
// Check c8ctl version and help
console.log('\n=== c8ctl Version ===');
c8('--version');

console.log('\n=== c8ctl Help ===');
c8('help');

In [ ]:
// Test cluster connectivity
console.log('\n=== Testing Cluster Connectivity ===');
try {
    c8('get topology');
    console.log('\n✓ Successfully connected to Camunda cluster at localhost:8080');
} catch (error) {
    console.error('\n✗ Failed to connect to Camunda cluster');
    console.error('Make sure Camunda 8 is running at localhost:8080');
    throw error;
}

## 2. Profile and Session Management

Configure and manage profiles for different environments.

In [ ]:
// Add a local profile (localhost:8080)
console.log('=== Adding Local Profile ===');
c8('add profile local --baseUrl=http://localhost:8080', { ignoreErrors: true });

console.log('\n=== List Profiles ===');
c8('list profiles');

console.log('\n=== Set Active Profile ===');
c8('use profile local');

console.log('\n✓ Profile configured for localhost');

In [ ]:
// Configure output mode for JSON (useful for scripting)
console.log('=== Setting Output Mode ===');
c8('output text');
console.log('✓ Output mode set to text (human-readable)');

## 3. Basic Deployment Operations

Deploy BPMN processes to the cluster.

In [ ]:
// Deploy a simple BPMN process
console.log('=== Deploying Simple Process ===');
const simpleBpmnPath = path.join(__dirname, '..', 'tests', 'fixtures', 'simple.bpmn');

if (!fs.existsSync(simpleBpmnPath)) {
    console.error('✗ Simple BPMN file not found:', simpleBpmnPath);
} else {
    c8(`deploy ${simpleBpmnPath}`);
    console.log('\n✓ Simple process deployed successfully');
}

In [ ]:
// Deploy a building block process (demonstrates _bb- priority)
console.log('=== Deploying Building Block Process ===');
const bbBpmnPath = path.join(__dirname, '..', 'tests', 'fixtures', '_bb-building-block', 'bb-process.bpmn');

if (!fs.existsSync(bbBpmnPath)) {
    console.error('✗ Building block BPMN file not found:', bbBpmnPath);
} else {
    c8(`deploy ${bbBpmnPath}`);
    console.log('\n✓ Building block process deployed successfully');
}

In [ ]:
// Deploy a directory with multiple resources
console.log('=== Deploying Sample Project Directory ===');
const projectDir = path.join(__dirname, '..', 'tests', 'fixtures', 'sample-project');

if (!fs.existsSync(projectDir)) {
    console.error('✗ Sample project directory not found:', projectDir);
} else {
    c8(`deploy ${projectDir}`);
    console.log('\n✓ Sample project deployed successfully');
    console.log('Note: Building blocks (_bb-*) are deployed first, then other files');
}

## 4. Process Instance Operations

Create, list, and manage process instances.

In [ ]:
// Create a process instance
console.log('=== Creating Process Instance ===');
const createOutput = c8('create pi --bpmnProcessId=simple-process', { silent: true });
console.log(createOutput);

// Extract process instance key from output (format: "Key: 123456")
const keyMatch = createOutput.match(/Key: (\d+)/);
let processInstanceKey = null;
if (keyMatch) {
    processInstanceKey = keyMatch[1];
    console.log(`\n✓ Process instance created with key: ${processInstanceKey}`);
} else {
    console.log('\n⚠️  Could not extract process instance key from output');
}

In [ ]:
// Create process instance with variables
console.log('=== Creating Process Instance with Variables ===');
const variables = JSON.stringify({ orderId: '12345', amount: 100.50, customer: 'John Doe' });
const createWithVarsOutput = c8(`create pi --bpmnProcessId=simple-process --variables='${variables}'`, { silent: true });
console.log(createWithVarsOutput);

const keyMatch2 = createWithVarsOutput.match(/Key: (\d+)/);
if (keyMatch2) {
    console.log(`\n✓ Process instance with variables created: ${keyMatch2[1]}`);
}

In [ ]:
// List all process instances
console.log('=== Listing Process Instances ===');
c8('list pi');
console.log('\n✓ Process instances listed');

In [ ]:
// Get specific process instance details
if (processInstanceKey) {
    console.log(`=== Getting Process Instance ${processInstanceKey} ===`);
    c8(`get pi ${processInstanceKey}`);
} else {
    console.log('⚠️  No process instance key available, skipping get operation');
}

In [ ]:
// Filter process instances by BPMN process ID
console.log('=== Filtering Process Instances by BPMN Process ID ===');
c8('list pi --bpmnProcessId=simple-process');
console.log('\n✓ Filtered process instances displayed');

## 5. User Task Operations

Work with user tasks in processes.

In [ ]:
// Deploy process with user task
console.log('=== Deploying Process with User Task ===');
const userTaskBpmnPath = path.join(__dirname, '..', 'tests', 'fixtures', 'list-pis', 'min-usertask.bpmn');

if (!fs.existsSync(userTaskBpmnPath)) {
    console.error('✗ User task BPMN file not found:', userTaskBpmnPath);
} else {
    c8(`deploy ${userTaskBpmnPath}`);
    console.log('\n✓ Process with user task deployed successfully');
}

In [ ]:
// Create instance of process with user task
console.log('=== Creating Process Instance with User Task ===');
const userTaskOutput = c8('create pi --bpmnProcessId=Process_0t60ay7', { silent: true });
console.log(userTaskOutput);

const utKeyMatch = userTaskOutput.match(/Key: (\d+)/);
let userTaskPiKey = null;
if (utKeyMatch) {
    userTaskPiKey = utKeyMatch[1];
    console.log(`\n✓ Process instance with user task created: ${userTaskPiKey}`);
}

In [ ]:
// Wait a moment for user task to be created
console.log('Waiting for user task to be available...');
await new Promise(resolve => setTimeout(resolve, 2000));
console.log('✓ Wait complete');

In [ ]:
// List user tasks
console.log('=== Listing User Tasks ===');
const userTasksList = c8('list ut', { silent: true });
console.log(userTasksList);

// Try to extract user task key
const lines = userTasksList.split('\n');
let userTaskKey = null;
for (const line of lines) {
    const match = line.match(/^\s*(\d{13,})/);
    if (match) {
        userTaskKey = match[1];
        console.log(`\n✓ Found user task with key: ${userTaskKey}`);
        break;
    }
}

if (!userTaskKey) {
    console.log('\n⚠️  No user tasks found (process may have completed instantly)');
}

In [ ]:
// Complete user task with variables
if (userTaskKey) {
    console.log(`=== Completing User Task ${userTaskKey} ===`);
    const taskVariables = JSON.stringify({ approved: true, notes: 'Approved via notebook', timestamp: new Date().toISOString() });
    c8(`complete ut ${userTaskKey} --variables='${taskVariables}'`);
    console.log('\n✓ User task completed successfully');
} else {
    console.log('⚠️  No user task key available, skipping completion');
}

## 6. Deploy and Run Workflow

The `run` command combines deployment and process instance creation.

In [ ]:
// Run command: deploy and create instance in one step
console.log('=== Running Process (Deploy + Create) ===');
const runVariables = JSON.stringify({ testRun: true, timestamp: new Date().toISOString() });
c8(`run ${simpleBpmnPath} --variables='${runVariables}'`);
console.log('\n✓ Process deployed and instance created via run command');

## 7. Message Correlation

Publish and correlate messages to running process instances.

In [ ]:
// Publish a message
console.log('=== Publishing Message ===');
const msgVariables = JSON.stringify({ status: 'completed', value: 42 });
c8(`publish msg order-placed --correlationKey=order-12345 --variables='${msgVariables}'`);
console.log('\n✓ Message published successfully');

In [ ]:
// Correlate message with time-to-live
console.log('=== Correlating Message with TTL ===');
c8('correlate msg payment-received --correlationKey=order-67890 --timeToLive=60000');
console.log('\n✓ Message correlated with 60s TTL');

## 8. Incident Management

List and resolve incidents that occur during process execution.

In [ ]:
// List incidents
console.log('=== Listing Incidents ===');
c8('list inc');
console.log('\n✓ Incidents listed (may be empty if no incidents exist)');

In [ ]:
// List active incidents only
console.log('=== Listing Active Incidents ===');
c8('list inc --state=ACTIVE');
console.log('\n✓ Active incidents filtered');

## 9. Job Activation and Completion

Work with jobs for service tasks.

In [ ]:
// List jobs
console.log('=== Listing Jobs ===');
c8('list jobs');
console.log('\n✓ Jobs listed');

In [ ]:
// Note: Job activation requires a specific job type
// This is a demonstration of the command syntax
console.log('=== Job Activation Example ===');
console.log('Command syntax: c8 activate jobs <type> --maxJobsToActivate=10 --timeout=60000');
console.log('Example: c8 activate jobs email-service --worker=my-worker');
console.log('\nNote: Actual activation requires jobs of the specified type to exist');

## 10. Plugin System

Extend c8ctl with custom plugins.

In [ ]:
// List installed plugins
console.log('=== Listing Plugins ===');
c8('list plugins');
console.log('\n✓ Plugins listed');

In [ ]:
// Load a test plugin from local directory
console.log('=== Loading Sample Plugin ===');
const pluginPath = path.join(__dirname, '..', 'tests', 'fixtures', 'plugins', 'js-plugin');

if (fs.existsSync(pluginPath)) {
    try {
        c8(`load plugin --from file://${pluginPath}`, { ignoreErrors: true });
        console.log('\n✓ Sample plugin loaded');
        
        // List plugins again to see the new one
        console.log('\n=== Updated Plugin List ===');
        c8('list plugins');
    } catch (error) {
        console.log('⚠️  Plugin loading not available or failed');
    }
} else {
    console.log('⚠️  Sample plugin directory not found');
}

## 11. Complete E2E Workflow

A comprehensive example combining multiple operations.

In [ ]:
console.log('=== Complete E2E Workflow ===\n');

// Step 1: Deploy
console.log('Step 1: Deploying process...');
c8(`deploy ${simpleBpmnPath}`);
console.log('✓ Deployed\n');

// Step 2: Create instance
console.log('Step 2: Creating process instance...');
const e2eVars = JSON.stringify({ 
    workflow: 'e2e-demo', 
    startTime: new Date().toISOString(),
    requester: 'notebook-user'
});
const e2eOutput = c8(`create pi --bpmnProcessId=simple-process --variables='${e2eVars}'`, { silent: true });
console.log(e2eOutput);
const e2eKey = e2eOutput.match(/Key: (\d+)/)?.[1];
console.log('✓ Created\n');

// Step 3: Monitor
if (e2eKey) {
    console.log('Step 3: Getting instance details...');
    c8(`get pi ${e2eKey}`);
    console.log('✓ Retrieved\n');
}

// Step 4: List all instances
console.log('Step 4: Listing all instances...');
c8('list pi --bpmnProcessId=simple-process');
console.log('✓ Listed\n');

console.log('\n✅ Complete E2E workflow executed successfully!');

## 12. Cleanup and Summary

View final state and cleanup resources.

In [ ]:
// Get final topology
console.log('=== Final Cluster Topology ===');
c8('get topology');
console.log('\n✓ Topology retrieved');

In [ ]:
// Summary of operations performed
console.log('\n=== Summary of Operations Performed ===\n');
console.log('✅ Environment Setup and Verification');
console.log('✅ Profile and Session Management');
console.log('✅ Basic Deployment Operations');
console.log('   - Simple process deployment');
console.log('   - Building block deployment');
console.log('   - Directory deployment');
console.log('✅ Process Instance Operations');
console.log('   - Create instances');
console.log('   - Create with variables');
console.log('   - List and filter instances');
console.log('   - Get specific instances');
console.log('✅ User Task Operations');
console.log('   - Deploy process with user task');
console.log('   - List user tasks');
console.log('   - Complete user task with variables');
console.log('✅ Deploy and Run Workflow');
console.log('✅ Message Correlation');
console.log('   - Publish messages');
console.log('   - Correlate with correlation key');
console.log('   - Set message TTL');
console.log('✅ Incident Management');
console.log('✅ Job Operations');
console.log('✅ Plugin System');
console.log('   - List plugins');
console.log('   - Load plugins');
console.log('✅ Complete E2E Workflow');
console.log('\n🎉 All operations completed successfully!');
console.log('\nFor more information, see:');
console.log('- README.md for general usage');
console.log('- EXAMPLES.md for more examples');
console.log('- PLUGIN-HELP.md for plugin development');